In [2]:
# import required libraries
import pandas as pd
import dash
from dash import html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px


# read the airline data into a pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


# create a dash application
app = dash.Dash(__name__)


# create an application layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center',
                                               'font-family' : 'Helvetica',
                                               'color': '#503D36',
                                               'font-size': 40,
                                               'backgroundColor' : 'white'
                                        }),
                                # TASK 1: add a dropdown list to enable Launch Site selection
                                # The default select value should be ALL sites
                                # note: TASK 1 has been completed successfully.
                                dcc.Dropdown(
                                    id = 'site-dropdown',
                                    options = [
                                        {'label' : 'All Sites', 'value' : 'ALL'},
                                        {'label' : 'CCAFS LC-40', 'value' : 'CCAFS LC-40'},
                                        {'label' : 'VAFB SLC-4E', 'value' : 'VAFB SLC-4E'},
                                        {'label' : 'KSC LC-39A', 'value' : 'KSC LC-39A'},
                                        {'label' : 'CCAFS SLC-40', 'value' : 'CCAFS SLC-40'},
                                        ],
                                    value = 'ALL',
                                    placeholder = 'Select a Launch Site Here',
                                    searchable = True
                                    ),
                                html.Br(),

                                # TASK 2: add a pie chart to show the total successful launch count for all sites
                                # if a specific launch site was selected, show the Success vs. Failed counts for the site
                                # note: TASK 2 has been completed successfully.
                                html.Div(
                                    dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: add a slider to select payload range
                                # note: TASK 3 has been completed successfully.
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min = 0,
                                    max = 10000,
                                    step = 1000,
                                    marks = {0 : '0',
                                            1000 : '1000',
                                            2000 : '2000',
                                            3000 : '3000',
                                            4000 : '4000',
                                            5000 : '5000',
                                            6000 : '6000',
                                            7000 : '7000',
                                            8000 : '8000',
                                            9000 : '9000',
                                            10000 : '10000'},
                                    value = [min_payload, max_payload]),

                                # TASK 4: add a scatter chart to show the correlation between payload and launch success
                                # note: TASK 4 has been completed successfully.
                                html.Div(
                                    dcc.Graph(id='success-payload-scatter-chart')),
                                html.Br()
                                ])

# TASK 2: add a callback function for site-dropdown as input and success-pie-chart as output
# note: TASK 2 has been completed successfully.
@app.callback(
    Output(component_id = 'success-pie-chart', component_property = 'figure'),
    Input(component_id = 'site-dropdown', component_property = 'value')
    )
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        data = {
            'Launch Site' : spacex_df['Launch Site'].unique().tolist(),
            'Total Launches': spacex_df['Launch Site'].value_counts().tolist(),
            }
        filtered_df = pd.DataFrame(data)
        fig = px.pie(
            filtered_df,
            values = 'Total Launches',
            names =  'Launch Site',
            title = 'Pie Chart Considering All Sites (Failed and Successfull Launches)'
            )
        return fig
    else:
        filtered_df = spacex_df.loc[spacex_df['Launch Site'] ==  entered_site]
        data = {
            'Launches' : filtered_df['class'].value_counts().tolist(),
            'Outcomes' : ['Failure', 'Success']
        }
        filtered_df = pd.DataFrame(data)
        fig = px.pie(
            filtered_df,
            values = 'Launches',
            names = 'Outcomes',
            title = 'Successful vs Failed Launch Attempts for ' + str(entered_site) + ' launch site'
            )
        return fig


# TASK 4: add a callback function for site-dropdown and payload-slider as inputs and success-payload-scatter-chart as output
@app.callback(
    Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
    Input(component_id = 'site-dropdown', component_property = 'value'),
    Input(component_id = 'payload-slider', component_property = 'value')
)
def get_payload_mass(entered_site, payload_range):
    if entered_site == 'ALL':
        data = {
            'Payload Mass (kg)' : spacex_df['Payload Mass (kg)'],
            'class' : spacex_df['class'],
            'Booster Version Category' : spacex_df['Booster Version Category']
        }
        filtered_df = pd.DataFrame(data)
        low, high = payload_range
        mask = (filtered_df['Payload Mass (kg)'] > low ) & (filtered_df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            filtered_df[mask],
            x = 'Payload Mass (kg)',
            y = 'class',
            title = 'Correlation between Payload Mass and Outcome for All Sites',
            color = 'Booster Version Category'
        )
        return fig
    else:
        filtered_df = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        data = {
            'Payload Mass (kg)' : spacex_df['Payload Mass (kg)'],
            'class' : spacex_df['class'],
            'Booster Version Category' : spacex_df['Booster Version Category']
        }
        filtered_df = pd.DataFrame(data)
        low, high = payload_range
        mask = (filtered_df['Payload Mass (kg)'] > low) & (filtered_df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            filtered_df[mask],
            x = 'Payload Mass (kg)',
            y = 'class',
            title = 'Correlation between Payload Mass and Outcome for ' + str(entered_site) + ' Launch Site',
            color = 'Booster Version Category'
        )
        return fig


# run the app
if __name__ == '__main__':
    app.run_server()